In [4]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('basic').getOrCreate()

In [5]:
from pyspark.sql.functions import (countDistinct,collect_set,collect_list,concat_ws,col,\
                nanvl,coalesce,abs,min,row_number,lead,lag,date_format,max,split,initcap,count,length,datediff,expr,avg,round,sum,avg,desc,asc,rank,when,expr,lit,to_date)
from pyspark.sql.types import StructType, StructField, IntegerType,StringType,DateType
from pyspark.sql.window import Window


#### 031 Managers with at Least 5 Direct Reports  

Table: Employee

+-------------+---------+
| Column Name | Type    |
+-------------+---------+
| id          | int     |
| name        | varchar |
| department  | varchar |
| managerId   | int     |
+-------------+---------+
id is the primary key (column with unique values) for this table.
Each row of this table indicates the name of an employee, their department, and the id of their manager.
If managerId is null, then the employee does not have a manager.
No employee will be the manager of themself.
 

Write a solution to find managers with at least five direct reports.

Return the result table in any order.

The result format is in the following example.

 

Example 1:

Input: 
Employee table:
+-----+-------+------------+-----------+
| id  | name  | department | managerId |
+-----+-------+------------+-----------+
| 101 | John  | A          | null      |
| 102 | Dan   | A          | 101       |
| 103 | James | A          | 101       |
| 104 | Amy   | A          | 101       |
| 105 | Anne  | A          | 101       |
| 106 | Ron   | B          | 101       |
+-----+-------+------------+-----------+
Output: 
+------+
| name |
+------+
| John |
+------+


In [6]:
mgr_df = spark.read.option('header',True).option('inferSchema',True).format('csv')\
        .load('../data/easymedium/031_ManagerswithatLeast5DirectReports.csv')

In [6]:
mgr_df.printSchema()

root
 |-- id: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- department: string (nullable = true)
 |-- managerId: integer (nullable = true)



In [24]:
 max_ge = mgr_df.groupby('managerId').agg(count('managerId').alias('mgr_cnt')).filter(col('mgr_cnt') >= 5)\
            .select(col('managerId').alias('max_mgr_id'),'mgr_cnt')

In [26]:
mgr_df.join(max_ge,col('max_mgr_id') == col('id'),'inner').select('name').show()

+----+
|name|
+----+
|John|
+----+



In [27]:
mgr_df.createOrReplaceTempView('mgrdf')

In [34]:
spark.sql("select name from mgrdf where id in \
            (select managerId from mgrdf group by managerId having count(managerId) >= 5 )").show()

+----+
|name|
+----+
|John|
+----+



#### 032 Confirmation Rate

Table: Signups

+----------------+----------+
| Column Name    | Type     |
+----------------+----------+
| user_id        | int      |
| time_stamp     | datetime |
+----------------+----------+
user_id is the column of unique values for this table.
Each row contains information about the signup time for the user with ID user_id.
 

Table: Confirmations

+----------------+----------+
| Column Name    | Type     |
+----------------+----------+
| user_id        | int      |
| time_stamp     | datetime |
| action         | ENUM     |
+----------------+----------+
(user_id, time_stamp) is the primary key (combination of columns with unique values) for this table.
user_id is a foreign key (reference column) to the Signups table.
action is an ENUM (category) of the type ('confirmed', 'timeout')
Each row of this table indicates that the user with ID user_id requested a confirmation message at time_stamp and that confirmation message was either confirmed ('confirmed') or expired without confirming ('timeout').
 

The confirmation rate of a user is the number of 'confirmed' messages divided by the total number of requested confirmation messages. The confirmation rate of a user that did not request any confirmation messages is 0. Round the confirmation rate to two decimal places.

Write a solution to find the confirmation rate of each user.

Return the result table in any order.

The result format is in the following example.

 

Example 1:

Input: 
Signups table:
+---------+---------------------+
| user_id | time_stamp          |
+---------+---------------------+
| 3       | 2020-03-21 10:16:13 |
| 7       | 2020-01-04 13:57:59 |
| 2       | 2020-07-29 23:09:44 |
| 6       | 2020-12-09 10:39:37 |
+---------+---------------------+
Confirmations table:
+---------+---------------------+-----------+
| user_id | time_stamp          | action    |
+---------+---------------------+-----------+
| 3       | 2021-01-06 03:30:46 | timeout   |
| 3       | 2021-07-14 14:00:00 | timeout   |
| 7       | 2021-06-12 11:57:29 | confirmed |
| 7       | 2021-06-13 12:58:28 | confirmed |
| 7       | 2021-06-14 13:59:27 | confirmed |
| 2       | 2021-01-22 00:00:00 | confirmed |
| 2       | 2021-02-28 23:59:59 | timeout   |
+---------+---------------------+-----------+
Output: 
+---------+-------------------+
| user_id | confirmation_rate |
+---------+-------------------+
| 6       | 0.00              |
| 3       | 0.00              |
| 7       | 1.00              |
| 2       | 0.50              |
+---------+-------------------+
Explanation: 
User 6 did not request any confirmation messages. The confirmation rate is 0.
User 3 made 2 requests and both timed out. The confirmation rate is 0.
User 7 made 3 requests and all were confirmed. The confirmation rate is 1.
User 2 made 2 requests where one was confirmed and the other timed out. The confirmation rate is 1 / 2 = 0.5.

In [7]:
signup_df = spark.read.option('header',True).option('inferSchema',True).format('csv')\
        .load('../data/easymedium/032_Confirmation_Rate.csv')

In [8]:
confirm_df = spark.read.option('header',True).option('inferSchema',True).format('csv')\
        .load('../data/easymedium/032a_Confirmation_Rate.csv')

In [39]:
signup_df.printSchema()

root
 |-- user_id: integer (nullable = true)
 |-- time_stamp: string (nullable = true)



In [48]:
confirm_df.printSchema()

root
 |-- user_id: integer (nullable = true)
 |-- time_stamp: string (nullable = true)
 |-- action: string (nullable = true)



In [89]:
join_df = signup_df.select(col('user_id').alias('sig_user_id'),col('time_stamp').alias('sig_timestamp'))\
        .join(confirm_df,col('sig_user_id') == col('user_id'),'left')

In [101]:
join_df.groupby('sig_user_id').agg( (sum(when(col('action') == 'confirmed',1).otherwise(0))/count('*')).alias('confirmation_rate') )\
                .show()

+-----------+-----------------+
|sig_user_id|confirmation_rate|
+-----------+-----------------+
|          6|              0.0|
|          3|              0.0|
|          7|              1.0|
|          2|              0.5|
+-----------+-----------------+



In [103]:
signup_df.createOrReplaceTempView('signupdf')
confirm_df.createOrReplaceTempView('confirmdf')

In [140]:
spark.sql(" select user_id,( sum(CASE WHEN action = 'confirmed' THEN 1 ELSE 0 END)/count(*) )as confirmation_rate  from \
    (select sig.user_id,con.action from signupdf sig left join confirmdf con on sig.user_id = con.user_id)as a  \
    GROUP BY a.user_id"  ) \
    .show()

+-------+-----------------+
|user_id|confirmation_rate|
+-------+-----------------+
|      6|              0.0|
|      3|              0.0|
|      7|              1.0|
|      2|              0.5|
+-------+-----------------+



#### 033 Monthly Transactions

Table: Transactions

+---------------+---------+
| Column Name   | Type    |
+---------------+---------+
| id            | int     |
| country       | varchar |
| state         | enum    |
| amount        | int     |
| trans_date    | date    |
+---------------+---------+
id is the primary key of this table.
The table has information about incoming transactions.
The state column is an enum of type ["approved", "declined"].
 

Write an SQL query to find for each month and country, the number of transactions and their total amount, the number of approved transactions and their total amount.

Return the result table in any order.

The query result format is in the following example.

 

Example 1:

Input: 
Transactions table:
+------+---------+----------+--------+------------+
| id   | country | state    | amount | trans_date |
+------+---------+----------+--------+------------+
| 121  | US      | approved | 1000   | 2018-12-18 |
| 122  | US      | declined | 2000   | 2018-12-19 |
| 123  | US      | approved | 2000   | 2019-01-01 |
| 124  | DE      | approved | 2000   | 2019-01-07 |
+------+---------+----------+--------+------------+
Output: 
+----------+---------+-------------+----------------+--------------------+-----------------------+
| month    | country | trans_count | approved_count | trans_total_amount | approved_total_amount |
+----------+---------+-------------+----------------+--------------------+-----------------------+
| 2018-12  | US      | 2           | 1              | 3000               | 1000                  |
| 2019-01  | US      | 1           | 1              | 2000               | 2000                  |
| 2019-01  | DE      | 1           | 1              | 2000               | 2000                  |
+----------+---------+-------------+----------------+--------------------+-----------------------+


In [9]:
trans_schema = StructType([
                StructField('id',IntegerType()),
                StructField('country',StringType()),
                StructField('state',StringType()),
                StructField('amount',IntegerType()),
                StructField('trans_date',DateType())
                ])

In [10]:
trans_df = spark.read.option('header',True).schema(trans_schema).format('csv')\
        .load('../data/easymedium/033_MonthlyTransactions.csv')

In [326]:
trans_df.printSchema()

root
 |-- id: integer (nullable = true)
 |-- country: string (nullable = true)
 |-- state: string (nullable = true)
 |-- amount: integer (nullable = true)
 |-- trans_date: date (nullable = true)



In [183]:
group_trans_df = trans_df.select(date_format(col('trans_date'),'yyyy-MM').alias('date'),'country','amount') \
        .groupby('date','country').agg(count('date').alias('num_trans'),sum('amount').alias('total_amount')) \
        .select('date','country','num_trans','total_amount',lit(0).alias('approved_count'),lit(0).alias('approved_total_amount') )

In [184]:
approved_trans_df = trans_df.filter(col('state') == 'approved')\
    .select(date_format(col('trans_date'),'yyyy-MM').alias('date'),'country','amount') \
    .groupby('date','country').agg(count('date').alias('approved_count'),sum('amount').alias('approved_total_amount')) \
    .select('date','country',lit(0).alias('num_trans'),lit(0).alias('total_amount'),'approved_count','approved_total_amount')

In [193]:
group_trans_df.union(approved_trans_df).groupby('date','country')\
    .agg(sum('num_trans').alias('num_trans'),\
    sum('total_amount').alias('total_amount'),sum('approved_count').alias('approved_count'), \
    sum('approved_total_amount').alias('approved_total_amount')).show()

+-------+-------+---------+------------+--------------+---------------------+
|   date|country|num_trans|total_amount|approved_count|approved_total_amount|
+-------+-------+---------+------------+--------------+---------------------+
|2019-01|     US|        1|        2000|             1|                 2000|
|2019-01|     DE|        1|        2000|             1|                 2000|
|2018-12|     US|        2|        3000|             1|                 1000|
+-------+-------+---------+------------+--------------+---------------------+



### No Union another method

In [301]:
#        count(when(col("state").isNotNull(), 1)).alias("num_trans"),
#       _sum(when(col("state").isNotNull(), col("amount"))).alias("total_amount"),

trans_df.withColumn('trans_date',date_format(col('trans_date'),'yyyy-MM')).groupBy('trans_date','country') \
    .agg( 
         count('trans_date').alias('num_trans'), \
         sum('amount').alias('total_amount'), \
         count(when(col('state') == 'approved',1)).alias('approved_count'), \
            sum(when(col('state') == 'approved',col('amount'))).alias('approvedd_count')\
         ).show()

+----------+-------+---------+------------+--------------+---------------+
|trans_date|country|num_trans|total_amount|approved_count|approvedd_count|
+----------+-------+---------+------------+--------------+---------------+
|   2019-01|     US|        1|        2000|             1|           2000|
|   2019-01|     DE|        1|        2000|             1|           2000|
|   2018-12|     US|        2|        3000|             1|           1000|
+----------+-------+---------+------------+--------------+---------------+



In [303]:
trans_df.createOrReplaceTempView('transdf')

In [237]:
spark.sql(" select trans_date,country,sum(num_trans) as num_trans,sum(total_amount) as total_amount \
            ,sum(approved_count) as approved_count,sum(approved_total_amount) as approved_total_amount from  \
              ( \
            (select date_format(trans_date,'yyyy/MM') as trans_date,country,count(*) as num_trans, sum(amount) as total_amount \
            ,0 as approved_count,0 as approved_total_amount from transdf group by trans_date,country) \
            UNION \
            (select date_format(trans_date,'yyyy/MM') as trans_date,country, 0 as num_trans, 0 as total_amount \
            ,count(trans_date) as approved_count, sum(amount) as approved_total_amount \
            from transdf where state = 'approved' group by trans_date,country \
            )  \
             ) group by trans_date,country ").show()

+----------+-------+---------+------------+--------------+---------------------+
|trans_date|country|num_trans|total_amount|approved_count|approved_total_amount|
+----------+-------+---------+------------+--------------+---------------------+
|   2018/12|     US|        2|        3000|             1|                 1000|
|   2019/01|     DE|        1|        2000|             1|                 2000|
|   2019/01|     US|        1|        2000|             1|                 2000|
+----------+-------+---------+------------+--------------+---------------------+



### No Union another method

In [319]:
spark.sql("select date_format(trans_date,'yyyy-MM') as date,country,count(trans_date) as num_trans, \
            sum(amount) as total_amount,count(IF(state = 'approved',1,0)) as approved_count, \
            sum(if(state = 'approved',amount,0)) as approved_total_amount \
            from transdf group by date,country").show()

+-------+-------+---------+------------+--------------+---------------------+
|   date|country|num_trans|total_amount|approved_count|approved_total_amount|
+-------+-------+---------+------------+--------------+---------------------+
|2019-01|     US|        1|        2000|             1|                 2000|
|2019-01|     DE|        1|        2000|             1|                 2000|
|2018-12|     US|        2|        3000|             2|                 1000|
+-------+-------+---------+------------+--------------+---------------------+



#### 034 Immediate Food Delivery

Table: Delivery

+-----------------------------+---------+
| Column Name                 | Type    |
+-----------------------------+---------+
| delivery_id                 | int     |
| customer_id                 | int     |
| order_date                  | date    |
| customer_pref_delivery_date | date    |
+-----------------------------+---------+
delivery_id is the column of unique values of this table.
The table holds information about food delivery to customers that make orders at some date and specify a preferred delivery date (on the same order date or after it).
 

If the customer's preferred delivery date is the same as the order date, then the order is called immediate; otherwise, it is called scheduled.

The first order of a customer is the order with the earliest order date that the customer made. It is guaranteed that a customer has precisely one first order.

Write a solution to find the percentage of immediate orders in the first orders of all customers, rounded to 2 decimal places.

The result format is in the following example.

 

Example 1:

Input: 
Delivery table:
+-------------+-------------+------------+-----------------------------+
| delivery_id | customer_id | order_date | customer_pref_delivery_date |
+-------------+-------------+------------+-----------------------------+
| 1           | 1           | 2019-08-01 | 2019-08-02                  |
| 2           | 2           | 2019-08-02 | 2019-08-02                  |
| 3           | 1           | 2019-08-11 | 2019-08-12                  |
| 4           | 3           | 2019-08-24 | 2019-08-24                  |
| 5           | 3           | 2019-08-21 | 2019-08-22                  |
| 6           | 2           | 2019-08-11 | 2019-08-13                  |
| 7           | 4           | 2019-08-09 | 2019-08-09                  |
+-------------+-------------+------------+-----------------------------+
Output: 
+----------------------+
| immediate_percentage |
+----------------------+
| 50.00                |
+----------------------+
Explanation: 
The customer id 1 has a first order with delivery id 1 and it is scheduled.
The customer id 2 has a first order with delivery id 2 and it is immediate.
The customer id 3 has a first order with delivery id 5 and it is scheduled.
The customer id 4 has a first order with delivery id 7 and it is immediate.
Hence, half the customers have immediate first orders.

In [11]:
delivery_schema = StructType([
                StructField('delivery_id',IntegerType()),
                StructField('customer_id',IntegerType()),
                StructField('order_date',DateType()),
                StructField('cust_prefdelivery_date',DateType()),
                ])

In [12]:
delivery_df = spark.read.option('header',True).schema(delivery_schema).format('csv')\
        .load('../data/easymedium/034_ImmediateFoodDelivery.csv')

In [343]:
delivery_df.printSchema()

root
 |-- delivery_id: integer (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- order_date: date (nullable = true)
 |-- cust_prefdelivery_date: date (nullable = true)



In [365]:
window_spec = Window.partitionBy('customer_id').orderBy('customer_id','order_date')

In [402]:
delivery_df.withColumn('diff',lag('order_date').over(window_spec)).filter(col('diff').isNull()) \
.withColumn('immediate',datediff(col('cust_prefdelivery_date'),col('order_date')))\
.select( (sum(when(col('immediate') == 0,1))/count('*')).alias('percentage') ).show()

+----------+
|percentage|
+----------+
|       0.5|
+----------+



In [403]:
delivery_df.createOrReplaceTempView('deliverydf')

In [475]:
spark.sql("select sum(if(flag = 0,1,0))/ count(*) as percentage from \
            (select delivery_id,customer_id,order_date,cust_prefdelivery_date, \
            lag(order_date) over(partition by customer_id order by customer_id,order_date)  as diff , \
            datediff(cust_prefdelivery_date,order_date) as flag \
            from deliverydf ) where diff is null \
              ").show()

+----------+
|percentage|
+----------+
|       0.5|
+----------+



In [464]:
spark.sql(" with first_orders as (select customer_id,min(order_date) as order_date \
    from deliverydf group by customer_id) \
    select sum(case when d.order_date = d.cust_prefdelivery_date then 1 else  0 end)/count(*)  as pct \
    from first_orders f inner join deliverydf d on d.customer_id = f.customer_id \
    and d.order_date = f.order_date \
    ").show()


+---+
|pct|
+---+
|0.5|
+---+



In [473]:
spark.sql(" with first_order as ( \
 select customer_id,order_date,cust_prefdelivery_date, \
 row_number() over(partition by customer_id order by customer_id,order_date) as rnum \
 from deliverydf ) \
     select sum(case when d.order_date = d.cust_prefdelivery_date then 1 else  0 end)/count(*) as pct \
     from first_order d where rnum = 1 \
         ").show()

+---+
|pct|
+---+
|0.5|
+---+



#### 035 Game Play Analysis

Table: Activity

+--------------+---------+
| Column Name  | Type    |
+--------------+---------+
| player_id    | int     |
| device_id    | int     |
| event_date   | date    |
| games_played | int     |
+--------------+---------+
(player_id, event_date) is the primary key (combination of columns with unique values) of this table.
This table shows the activity of players of some games.
Each row is a record of a player who logged in and played a number of games (possibly 0) before logging out on someday using some device.
 

Write a solution to report the fraction of players that logged in again on the day after the day they first logged in, rounded to 2 decimal places. In other words, you need to count the number of players that logged in for at least two consecutive days starting from their first login date, then divide that number by the total number of players.

The result format is in the following example.

 

Example 1:

Input: 
Activity table:
+-----------+-----------+------------+--------------+
| player_id | device_id | event_date | games_played |
+-----------+-----------+------------+--------------+
| 1         | 2         | 2016-03-01 | 5            |
| 1         | 2         | 2016-03-02 | 6            |
| 2         | 3         | 2017-06-25 | 1            |
| 3         | 1         | 2016-03-02 | 0            |
| 3         | 4         | 2018-07-03 | 5            |
+-----------+-----------+------------+--------------+
Output: 
+-----------+
| fraction  |
+-----------+
| 0.33      |
+-----------+
Explanation: 
Only the player with id 1 logged back in after the first day he had logged in so the answer is 1/3 = 0.33

In [13]:
game_schema = StructType([
                StructField('player_id',IntegerType()),
                StructField('device_id',IntegerType()),
                StructField('event_date',DateType()),
                StructField('games_played',IntegerType()),
                ])

In [14]:
game_df = spark.read.option('header',True).schema(game_schema).format('csv')\
        .load('../data/easymedium/035_GamePlayAnalysis.csv')

In [110]:
game_df.printSchema()

root
 |-- player_id: integer (nullable = true)
 |-- device_id: integer (nullable = true)
 |-- event_date: date (nullable = true)
 |-- games_played: integer (nullable = true)



In [126]:
game_df.groupby('player_id').agg(when(datediff(max(col('event_date')),\
                            min(col('event_date'))) == 1,1).otherwise(0).alias('x')) \
                            .select(round(sum(when(col('x') == 1 ,1).otherwise(0))/count('*'),2).alias('franction')) \
                            .show()

+---------+
|franction|
+---------+
|     0.33|
+---------+



In [106]:
game_window = Window.partitionBy('player_id').orderBy('player_id','event_date')

In [167]:
game_df.createOrReplaceTempView('gamedf')

In [85]:
spark.sql("select round(sum(if(a.x=1,1,0))/count(*),2) as fraction from \
          (select player_id,if(datediff( max(event_date),min(event_date)) = 1,1,0) as x\
          from gamedf group by player_id ) as a ").show() 

+--------+
|fraction|
+--------+
|    0.33|
+--------+



#### 036 Product Sales Analysis

Table: Sales

+-------------+-------+
| Column Name | Type  |
+-------------+-------+
| sale_id     | int   |
| product_id  | int   |
| year        | int   |
| quantity    | int   |
| price       | int   |
+-------------+-------+
(sale_id, year) is the primary key (combination of columns with unique values) of this table.
product_id is a foreign key (reference column) to Product table.
Each row of this table shows a sale on the product product_id in a certain year.
Note that the price is per unit.
 

Table: Product

+--------------+---------+
| Column Name  | Type    |
+--------------+---------+
| product_id   | int     |
| product_name | varchar |
+--------------+---------+
product_id is the primary key (column with unique values) of this table.
Each row of this table indicates the product name of each product.
 

Write a solution to select the product id, year, quantity, and price for the first year of every product sold.

Return the resulting table in any order.

The result format is in the following example.

 

Example 1:

Input: 
Sales table:
+---------+------------+------+----------+-------+
| sale_id | product_id | year | quantity | price |
+---------+------------+------+----------+-------+ 
| 1       | 100        | 2008 | 10       | 5000  |
| 2       | 100        | 2009 | 12       | 5000  |
| 7       | 200        | 2011 | 15       | 9000  |
+---------+------------+------+----------+-------+
Product table:
+------------+--------------+
| product_id | product_name |
+------------+--------------+
| 100        | Nokia        |
| 200        | Apple        |
| 300        | Samsung      |
+------------+--------------+
Output: 
+------------+------------+----------+-------+
| product_id | first_year | quantity | price |
+------------+------------+----------+-------+ 
| 100        | 2008       | 10       | 5000  |
| 200        | 2011       | 15       | 9000  |
+------------+------------+----------+-------+

In [15]:
sles_df = spark.read.option('header',True).option('inferSchema',True).format('csv')\
        .load('../data/easymedium/036_ProductSalesAnalysis.csv')


In [16]:
product_df = spark.read.option('header',True).option('inferSchema',True).format('csv')\
        .load('../data/easymedium/036a_ProductSalesAnalysis.csv')

In [143]:
sles_df.printSchema()

root
 |-- sale_id: integer (nullable = true)
 |-- product_id: integer (nullable = true)
 |-- year: integer (nullable = true)
 |-- quantity: integer (nullable = true)
 |-- price: integer (nullable = true)



In [144]:
product_df.printSchema()

root
 |-- product_id: integer (nullable = true)
 |-- product_name: string (nullable = true)



In [166]:
sles_df.groupby('product_id').agg(min(col('year')).alias('first_year'))\
    .select(col('product_id').alias('agg_product_id'),col('first_year')) \
    .join(sles_df,(col('first_year') == col('year')) & (col('agg_product_id') == col('product_id')),'inner') \
    .select('product_id','first_year','quantity','price') \
    .show()

+----------+----------+--------+-----+
|product_id|first_year|quantity|price|
+----------+----------+--------+-----+
|       100|      2008|      10| 5000|
|       200|      2011|      15| 9000|
+----------+----------+--------+-----+



In [168]:
sles_df.createOrReplaceTempView('slesdf')

In [180]:
spark.sql(" select product_id,a.first_year,quantity,price from slesdf join \
            (select product_id as agg_product_id, min(year) as first_year from slesdf group by product_id) a \
            on a.agg_product_id = product_id and a.first_year = year ").show()

+----------+----------+--------+-----+
|product_id|first_year|quantity|price|
+----------+----------+--------+-----+
|       100|      2008|      10| 5000|
|       200|      2011|      15| 9000|
+----------+----------+--------+-----+



#### 037 Customers Who Bought All Products  

Table: Customer

+-------------+---------+
| Column Name | Type    |
+-------------+---------+
| customer_id | int     |
| product_key | int     |
+-------------+---------+
This table may contain duplicates rows. 
customer_id is not NULL.
product_key is a foreign key (reference column) to Product table.
 

Table: Product

+-------------+---------+
| Column Name | Type    |
+-------------+---------+
| product_key | int     |
+-------------+---------+
product_key is the primary key (column with unique values) for this table.
 

Write a solution to report the customer ids from the Customer table that bought all the products in the Product table.

Return the result table in any order.

The result format is in the following example.

 

Example 1:

Input: 
Customer table:
+-------------+-------------+
| customer_id | product_key |
+-------------+-------------+
| 1           | 5           |
| 2           | 6           |
| 3           | 5           |
| 3           | 6           |
| 1           | 6           |
+-------------+-------------+
Product table:
+-------------+
| product_key |
+-------------+
| 5           |
| 6           |
+-------------+
Output: 
+-------------+
| customer_id |
+-------------+
| 1           |
| 3           |
+-------------+
Explanation: 
The customers who bought all the products (5 and 6) are customers with IDs 1 and 3.

In [17]:
customer_df = spark.read.option('header',True).option('inferSchema',True).format('csv')\
        .load('../data/easymedium/037_CustomersWhoBoughtAllProducts.csv')

In [18]:
product_df = spark.read.option('header',True).option('inferSchema',True).format('csv')\
        .load('../data/easymedium/037a_CustomersWhoBoughtAllProducts.csv')


In [251]:
customer_df.printSchema()

root
 |-- customer_id: integer (nullable = true)
 |-- product_key: integer (nullable = true)



In [281]:
product_df.printSchema()

root
 |-- product_key: integer (nullable = true)



In [286]:
customer_df.groupby('customer_id').agg(count('product_key').alias('cnt_key')) \
      .join(product_df,product_df.select(count('*')).collect()[0][0] == col('cnt_key'),'inner') \
      .select('customer_id').distinct().show()

+-----------+
|customer_id|
+-----------+
|          1|
|          3|
+-----------+



In [255]:
customer_df.createOrReplaceTempView('customerdf')
product_df.createOrReplaceTempView('productdf')

In [272]:
spark.sql(" select customer_id from \
    ( select c.customer_id,count(c.product_key) as c_key from customerdf c join productdf p on  c.product_key \
    = p.product_key group by c.customer_id \
    having c_key in (select count(*) from productdf ) )").show()

+-----------+
|customer_id|
+-----------+
|          1|
|          3|
+-----------+



#### 038 Consecutive Numbers

Table: Logs

+-------------+---------+
| Column Name | Type    |
+-------------+---------+
| id          | int     |
| num         | varchar |
+-------------+---------+
In SQL, id is the primary key for this table.
id is an autoincrement column.
 

Find all numbers that appear at least three times consecutively.

Return the result table in any order.

The result format is in the following example.

 

Example 1:

Input: 
Logs table:
+----+-----+
| id | num |
+----+-----+
| 1  | 1   |
| 2  | 1   |
| 3  | 1   |
| 4  | 2   |
| 5  | 1   |
| 6  | 2   |
| 7  | 2   |
+----+-----+
Output: 
+-----------------+
| ConsecutiveNums |
+-----------------+
| 1               |
+-----------------+
Explanation: 1 is the only number that appears consecutively for at least three times.

In [19]:
consecutive_df = spark.read.option('header',True).option('inferSchema',True).format('csv')\
        .load('../data/easymedium/038_ConsecutiveNumbers.csv')

In [73]:
consecutive_df.printSchema()

root
 |-- id: integer (nullable = true)
 |-- num: integer (nullable = true)



In [74]:
consecutive_df.show()

+---+---+
| id|num|
+---+---+
|  1|  1|
|  2|  1|
|  3|  1|
|  4|  2|
|  5|  2|
|  6|  2|
|  7|  2|
+---+---+



In [6]:
game_window = Window.orderBy('id')

In [7]:
xxx = consecutive_df.select( lag('num').over(game_window).alias('lags'),col('num')) 

In [78]:
xxx.select('num','lags',when( (col('lags').isNull()),consecutive_df.select('num').collect()[0]['num'])\
           .when(col('num') == col('lags'),col('lags')).otherwise(0).alias('x'))\
           .select(when( (col('num') == col('x')),col('x')).alias('con_num')).groupby(col('con_num'))\
           .agg(count(col('con_num')).alias('ccnt')).filter(col('ccnt') >= 3).select('con_num').show()

+-------+
|con_num|
+-------+
|      1|
|      2|
+-------+



24/07/22 12:25:25 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


In [75]:
consecutive_df.createOrReplaceTempView('consecutivedf')

In [77]:
spark.sql(" with lags as (select lag(num) over(order by id) as lags,num from consecutivedf) \
            select CASE WHEN lags is null THEN  ( select num from consecutivedf limit 1)  \
                        WHEN lags  = num THEN num  else 0 END  as x from lags \
                  GROUP BY x \
                  HAVING count(*) >=3  and x !=0\
                  ").show()

+---+
|  x|
+---+
|  1|
|  2|
+---+



24/07/22 12:25:10 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


#### 039 Product Price at a Given Date

Table: Products

+---------------+---------+
| Column Name   | Type    |
+---------------+---------+
| product_id    | int     |
| new_price     | int     |
| change_date   | date    |
+---------------+---------+
(product_id, change_date) is the primary key (combination of columns with unique values) of this table.
Each row of this table indicates that the price of some product was changed to a new price at some date.
 

Write a solution to find the prices of all products on 2019-08-16. Assume the price of all products before any change is 10.

Return the result table in any order.

The result format is in the following example.

 

Example 1:

Input: 
Products table:
+------------+-----------+-------------+
| product_id | new_price | change_date |
+------------+-----------+-------------+
| 1          | 20        | 2019-08-14  |
| 2          | 50        | 2019-08-14  |
| 1          | 30        | 2019-08-15  |
| 1          | 35        | 2019-08-16  |
| 2          | 65        | 2019-08-17  |
| 3          | 20        | 2019-08-18  |
+------------+-----------+-------------+
Output: 
+------------+-------+
| product_id | price |
+------------+-------+
| 2          | 50    |
| 1          | 35    |
| 3          | 10    |
+------------+-------+

In [20]:
price_schema = StructType([
                StructField('product_id',IntegerType()),
                StructField('new_price',IntegerType()),
                StructField('change_date',DateType()),
                ])

In [21]:
price_df = spark.read.option('header',True).option('inferSchema',True).format('csv')\
        .load('../data/easymedium/039_ProductPriceataGivenDate.csv')

In [269]:
price_df.printSchema()

root
 |-- product_id: integer (nullable = true)
 |-- new_price: integer (nullable = true)
 |-- change_date: string (nullable = true)



In [84]:
price_df.show()

+----------+---------+-----------+
|product_id|new_price|change_date|
+----------+---------+-----------+
|         1|       20| 2019-08-14|
|         2|       50| 2019-08-14|
|         1|       30| 2019-08-15|
|         1|       35| 2019-08-16|
|         2|       65| 2019-08-17|
|         3|       20| 2019-08-18|
+----------+---------+-----------+



In [85]:
window_key = Window.partitionBy(col('product_id')).orderBy(col('change_date'))

In [94]:
price_df.select('product_id','new_price','change_date',rank().over(window_key).alias('rnk'))\
                    .orderBy(col('product_id'),col('rnk'))

+----------+---------+-----------+---+
|product_id|new_price|change_date|rnk|
+----------+---------+-----------+---+
|         1|       20| 2019-08-14|  1|
|         1|       30| 2019-08-15|  2|
|         1|       35| 2019-08-16|  3|
|         2|       50| 2019-08-14|  1|
|         2|       65| 2019-08-17|  2|
|         3|       20| 2019-08-18|  1|
+----------+---------+-----------+---+



In [96]:
price_diff = (price_df.filter( (col('product_id') == 1 )& (col('change_date') == lit('2019-08-16')) )\
        .select(col('new_price')).collect()[0][0]) - (price_df.filter( (col('product_id') == 1 )& (col('change_date') == lit('2019-08-15')) ) \
    .select(col('new_price')).collect()[0][0])

In [175]:
price_df.select('product_id','new_price','change_date',(abs(datediff(col('change_date'),lit('2019-08-16')))\
          * price_diff).alias('datediff')).withColumn('price',(col('new_price')-col('datediff')))\
            .filter(col('change_date') >= lit('2019-08-16')).select('product_id','price').show()

+----------+-----+
|product_id|price|
+----------+-----+
|         1|   35|
|         2|   60|
|         3|   10|
+----------+-----+



In [270]:
price_df.createOrReplaceTempView('pricedf')

In [265]:
spark.sql("with fil_product_id as ( \
                        select distinct product_id,max(change_date) as change_date \
                        from pricedf where change_date <= '2019-08-16' \
                        group by product_id), \
                price_product_id as ( \
                        select y.product_id,x.new_price,x.change_date \
                        from pricedf x \
                        join fil_product_id y on x.product_id = y.product_id  and  x.change_date = y.change_date )\
                select distinct p.product_id, pp.new_price from  from pricedf p \
                        left join price_product_id pp on p.product_id = pp.product_id \
        ").show()



+----------+----+
|product_id|from|
+----------+----+
|         3|null|
|         2|  50|
|         1|  35|
+----------+----+



In [278]:
spark.sql(" (SELECT product_id, 10 as price FROM pricedf \
            GROUP BY product_id HAVING  min(change_date) > '2019-08-16') \
            UNION  \
            (SELECT product_id,new_price AS price FROM pricedf WHERE (product_id, change_date) IN ( \
            SELECT product_id, MAX(change_date) FROM pricedf WHERE change_date <= '2019-08-16' GROUP BY product_id)) \
    ").show()

+----------+-----+
|product_id|price|
+----------+-----+
|         3|   10|
|         2|   50|
|         1|   35|
+----------+-----+



#### 040 Last Person to Fit in the Bus

Table: Queue

+-------------+---------+
| Column Name | Type    |
+-------------+---------+
| person_id   | int     |
| person_name | varchar |
| weight      | int     |
| turn        | int     |
+-------------+---------+
person_id column contains unique values.
This table has the information about all people waiting for a bus.
The person_id and turn columns will contain all numbers from 1 to n, where n is the number of rows in the table.
turn determines the order of which the people will board the bus, where turn=1 denotes the first person to board and turn=n denotes the last person to board.
weight is the weight of the person in kilograms.
 

There is a queue of people waiting to board a bus. However, the bus has a weight limit of 1000 kilograms, so there may be some people who cannot board.

Write a solution to find the person_name of the last person that can fit on the bus without exceeding the weight limit. The test cases are generated such that the first person does not exceed the weight limit.

The result format is in the following example.

 

Example 1:

Input: 
Queue table:
+-----------+-------------+--------+------+
| person_id | person_name | weight | turn |
+-----------+-------------+--------+------+
| 5         | Alice       | 250    | 1    |
| 4         | Bob         | 175    | 5    |
| 3         | Alex        | 350    | 2    |
| 6         | John Cena   | 400    | 3    |
| 1         | Winston     | 500    | 6    |
| 2         | Marie       | 200    | 4    |
+-----------+-------------+--------+------+
Output: 
+-------------+
| person_name |
+-------------+
| John Cena   |
+-------------+
Explanation: The folowing table is ordered by the turn for simplicity.
+------+----+-----------+--------+--------------+
| Turn | ID | Name      | Weight | Total Weight |
+------+----+-----------+--------+--------------+
| 1    | 5  | Alice     | 250    | 250          |
| 2    | 3  | Alex      | 350    | 600          |
| 3    | 6  | John Cena | 400    | 1000         | (last person to board)
| 4    | 2  | Marie     | 200    | 1200         | (cannot board)
| 5    | 4  | Bob       | 175    | ___          |
| 6    | 1  | Winston   | 500    | ___          |
+------+----+-----------+--------+--------------+

In [22]:
fitness_df = price_df = spark.read.option('header',True).option('inferSchema',True).format('csv')\
        .load('../data/easymedium/040_LastPersontoFitintheBus.csv')

In [284]:
fitness_df.printSchema()

root
 |-- person_id: integer (nullable = true)
 |-- person_name: string (nullable = true)
 |-- weight: integer (nullable = true)
 |-- turn: integer (nullable = true)



In [287]:
fitness_df.select('turn','person_id','person_name','weight').sort('turn').show()

+----+---------+-----------+------+
|turn|person_id|person_name|weight|
+----+---------+-----------+------+
|   1|        5|      Alice|   250|
|   2|        3|       Alex|   350|
|   3|        6|  John Cena|   400|
|   4|        2|      Marie|   200|
|   5|        4|        Bob|   175|
|   6|        1|    Winston|   500|
+----+---------+-----------+------+



In [338]:
window_spec_x= Window.orderBy("turn").rowsBetween(Window.unboundedPreceding,Window.currentRow)

In [343]:
fitness_df.select('turn','person_id','person_name','weight').sort('turn') \
        .withColumn('rollweight',sum('weight').over(window_spec_x)).filter(col('rollweight') ==1000).select('person_name').show()

+-----------+
|person_name|
+-----------+
|  John Cena|
+-----------+



24/07/22 23:20:05 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


In [344]:
fitness_df.createOrReplaceTempView('fitnessdf')

In [359]:
spark.sql("with main as (select turn,person_id,person_name,weight from fitnessdf order by turn) \
          select person_name from \
          (select person_name,sum(weight) over(order by turn rows between unbounded preceding and current row) \
          as cumulative_sum \
          from main) as y \
          where cumulative_sum = 1000").show()

+-----------+
|person_name|
+-----------+
|  John Cena|
+-----------+



24/07/22 23:36:40 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


#### 041 Count Salary Categories

Table: Accounts

+-------------+------+
| Column Name | Type |
+-------------+------+
| account_id  | int  |
| income      | int  |
+-------------+------+
account_id is the primary key (column with unique values) for this table.
Each row contains information about the monthly income for one bank account.
 

Write a solution to calculate the number of bank accounts for each salary category. The salary categories are:

"Low Salary": All the salaries strictly less than $20000.
"Average Salary": All the salaries in the inclusive range [$20000, $50000].
"High Salary": All the salaries strictly greater than $50000.
The result table must contain all three categories. If there are no accounts in a category, return 0.

Return the result table in any order.

The result format is in the following example.

 

Example 1:

Input: 
Accounts table:
+------------+--------+
| account_id | income |
+------------+--------+
| 3          | 108939 |
| 2          | 12747  |
| 8          | 87709  |
| 6          | 91796  |
+------------+--------+
Output: 
+----------------+----------------+
| category       | accounts_count |
+----------------+----------------+
| Low Salary     | 1              |
| Average Salary | 0              |
| High Salary    | 3              |
+----------------+----------------+
Explanation: 
Low Salary: Account 2.
Average Salary: No accounts.
High Salary: Accounts 3, 6, and 8.

In [23]:
cntcat_df = price_df = spark.read.option('header',True).option('inferSchema',True).format('csv')\
        .load('../data/easymedium/041_CountSalaryCategories.csv')

In [362]:
cntcat_df.printSchema()

root
 |-- account_id: integer (nullable = true)
 |-- income: integer (nullable = true)



In [5]:
cntcat_df.show()

+----------+------+
|account_id|income|
+----------+------+
|         3|108939|
|         2| 12747|
|         8| 87709|
|         6| 91796|
+----------+------+



In [55]:
x = (20000,50000)
cntcat_df.filter(  (cntcat_df['income'] > 20000) & (cntcat_df['income'] <= 50000) )\
    .agg(count('account_id').alias('cnt')).select(col('cnt'),lit('Average Salary').alias('category')).show()


+---+--------------+
|cnt|      category|
+---+--------------+
|  0|Average Salary|
+---+--------------+



In [59]:
cntcat_df.filter( cntcat_df['income'].between(*x) ).select(count('*').alias('cnt'),lit('Average Salary').alias('category')).show()


+---+--------------+
|cnt|      category|
+---+--------------+
|  0|Average Salary|
+---+--------------+



In [73]:
cntcat_df.filter(col('income') < 20000).groupby('account_id').agg(count('account_id').alias('cnt'))\
        .select(col('cnt'),lit('Low Salary').alias('category'))\
        .union(cntcat_df.filter( cntcat_df['income'].between(*x) ).select(count('*').alias('cnt'),lit('Average Salary').alias('category'))) \
        .union(cntcat_df.filter( col('income') > 50000 ).select(count('*').alias('cnt'),lit('High Salary').alias('category'))) \
        .select('category','cnt').show()


+--------------+---+
|      category|cnt|
+--------------+---+
|    Low Salary|  1|
|Average Salary|  0|
|   High Salary|  3|
+--------------+---+



In [74]:
cntcat_df.createOrReplaceTempView('cntcatdf')

In [183]:
spark.sql(" (select 'Low Salary' as category,coalesce(sum(cnt),0) as cnt from \
            (select  count(*) as cnt from cntcatdf where income < 20000 group by account_id) group by cnt )\
    union(select 'Average Salary' as category,sum(cnt) as cnt from \
            (select case when income > 20000  and income  <= 50000 then count(*) else 0 end as cnt from cntcatdf group by income) \
            group by cnt) \
    union(select 'High Salary',sum(cnt) as cnt from \
          (select count(*) as cnt from cntcatdf where income > 50000 group by account_id) group by cnt) \
          ").show()

+--------------+---+
|      category|cnt|
+--------------+---+
|   High Salary|  3|
|Average Salary|  0|
|    Low Salary|  1|
+--------------+---+



#### 042 Exchange Seats

Table: Seat

+-------------+---------+
| Column Name | Type    |
+-------------+---------+
| id          | int     |
| student     | varchar |
+-------------+---------+
id is the primary key (unique value) column for this table.
Each row of this table indicates the name and the ID of a student.
id is a continuous increment.
 

Write a solution to swap the seat id of every two consecutive students. If the number of students is odd, the id of the last student is not swapped.

Return the result table ordered by id in ascending order.

The result format is in the following example.

 

Example 1:

Input: 
Seat table:
+----+---------+
| id | student |
+----+---------+
| 1  | Abbot   |
| 2  | Doris   |
| 3  | Emerson |
| 4  | Green   |
| 5  | Jeames  |
+----+---------+
Output: 
+----+---------+
| id | student |
+----+---------+
| 1  | Doris   |
| 2  | Abbot   |
| 3  | Green   |
| 4  | Emerson |
| 5  | Jeames  |
+----+---------+
Explanation: 
Note that if the number of students is odd, there is no need to change the last one's seat.

In [24]:
exchange_df = price_df = spark.read.option('header',True).option('inferSchema',True).format('csv')\
        .load('../data/easymedium/042_ExchangeSeats.csv')

In [581]:
exchange_df.printSchema()

root
 |-- id: integer (nullable = true)
 |-- student: string (nullable = true)



In [582]:
exchange_df.show()

+---+-------+
| id|student|
+---+-------+
|  1|  Abbot|
|  2|  Doris|
|  3|Emerson|
|  4|  Green|
|  5| Jeames|
|  6|  qqqqq|
+---+-------+



In [583]:
parity = exchange_df.select(max('id')).collect()[0][0]

In [561]:
odd = exchange_df.filter(col('id') < parity ) \
    .filter(col('id')%2 == 0).select((col('id')-1).alias('id'),'student').union(\
     exchange_df.filter(col('id') < parity ) \
    .filter(col('id')%2 != 0).select((col('id')+1).alias('id'),'student'))\
    .union(exchange_df.filter(col('id') == parity)) \
    .sort('id')

In [562]:
even = exchange_df.filter(col('id')%2 != 0).select((col('id')+1).alias('id'),'student')\
.union(exchange_df.filter(col('id')%2 == 0).select((col('id')-1).alias('id'),'student')).sort('id')

In [563]:
if parity %2  == 1 :
    odd.show()
else:
    even.show()
    

+---+-------+
| id|student|
+---+-------+
|  1|  Doris|
|  2|  Abbot|
|  3|  Green|
|  4|Emerson|
|  5| Jeames|
+---+-------+



In [558]:
exchange_df.createOrReplaceTempView('exchangedf')

In [559]:
spark.sql("with parity as (select max(id) as maxid from exchangedf), \
                   odd as (select  * from exchangedf where id  < (select maxid from parity)),\
                  oddx as (select * from ( \
                          (select id-1 as id ,student from odd where id%2 = 0) union \
                          (select id+1 as id ,student from odd where id%2 != 0) union \
                          (select id,student from exchangedf where id = (select maxid from parity)) order by id )),\
                 evenx as (select * from ( \
                          (select id-1 as id ,student from exchangedf where id%2 = 0) union \
                          (select id+1 as id ,student from exchangedf where id%2 != 0) order by id)) \
                 select id,student from \
                    (select *,'oddx' as source from oddx union select *,'evenx' as source from evenx) combined \
                  where source =(select case when maxid%2 = 1 then 'oddx' else 'evenx' end from parity) order by id \
                    ").show()            


+---+-------+
| id|student|
+---+-------+
|  1|  Doris|
|  2|  Abbot|
|  3|  Green|
|  4|Emerson|
|  5| Jeames|
+---+-------+



In [571]:
spark.sql("with parity as (select max(id) as maxid from exchangedf), \
                   odd as (select  * from exchangedf where id  < (select maxid from parity)),\
                  oddx as ( \
                          select id-1 as id ,student from odd where id%2 = 0 union \
                          select id+1 as id ,student from odd where id%2 != 0 union \
                          select id,student from exchangedf where id = (select maxid from parity) order by id ),\
                 evenx as ( \
                          select id-1 as id ,student from exchangedf where id%2 = 0 union \
                          select id+1 as id ,student from exchangedf where id%2 != 0 order by id) \
                 select id,student from \
                    (select *,'oddx' as source from oddx union select *,'evenx' as source from evenx) combined \
                  where source =(select case when maxid%2 = 1 then 'oddx' else 'evenx' end from parity) order by id \
                    ").show()            


+---+-------+
| id|student|
+---+-------+
|  1|  Doris|
|  2|  Abbot|
|  3|  Green|
|  4|Emerson|
|  5| Jeames|
+---+-------+



In [577]:
exchange_df.createOrReplaceTempView('exchangedf')

SELECT
    s1.id, COALESCE(s2.student, s1.student) AS student
FROM
    seat s1
        LEFT JOIN
    seat s2 ON ((s1.id + 1) ^ 1) - 1 = s2.id
ORDER BY s1.id;

SELECT
    (CASE
        WHEN MOD(id, 2) != 0 AND counts != id THEN id + 1
        WHEN MOD(id, 2) != 0 AND counts = id THEN id
        ELSE id - 1
    END) AS id,
    student
FROM
    seat,
    (SELECT
        COUNT(*) AS counts
    FROM
        seat) AS seat_counts
ORDER BY id ASC


In [584]:
query  = """
SELECT
    (CASE
        WHEN MOD(id, 2) != 0 AND counts != id THEN id + 1
        WHEN MOD(id, 2) != 0 AND counts = id THEN id
        ELSE id - 1
    END) AS id,
    student
FROM
    exchangedf,
    (SELECT
        COUNT(*) AS counts
    FROM
        exchangedf) AS seat_counts
ORDER BY id ASC
"""

In [596]:
spark.sql("select student,x.counts,id,id%2 as mod from exchangedf,   \
          (SELECT COUNT(*) AS counts FROM exchangedf)as x ").show()

+-------+------+---+---+
|student|counts| id|mod|
+-------+------+---+---+
|  Abbot|     5|  1|  1|
|  Doris|     5|  2|  0|
|Emerson|     5|  3|  1|
|  Green|     5|  4|  0|
| Jeames|     5|  5|  1|
+-------+------+---+---+



In [585]:
spark.sql(query).show()

+---+-------+
| id|student|
+---+-------+
|  1|  Doris|
|  2|  Abbot|
|  3|  Green|
|  4|Emerson|
|  5|  qqqqq|
|  6| Jeames|
+---+-------+



#### 043_Movie Rating

Table: Movies

+---------------+---------+
| Column Name   | Type    |
+---------------+---------+
| movie_id      | int     |
| title         | varchar |
+---------------+---------+
movie_id is the primary key (column with unique values) for this table.
title is the name of the movie.
 

Table: Users

+---------------+---------+
| Column Name   | Type    |
+---------------+---------+
| user_id       | int     |
| name          | varchar |
+---------------+---------+
user_id is the primary key (column with unique values) for this table.
 

Table: MovieRating

+---------------+---------+
| Column Name   | Type    |
+---------------+---------+
| movie_id      | int     |
| user_id       | int     |
| rating        | int     |
| created_at    | date    |
+---------------+---------+
(movie_id, user_id) is the primary key (column with unique values) for this table.
This table contains the rating of a movie by a user in their review.
created_at is the user's review date. 
 

Write a solution to:

Find the name of the user who has rated the greatest number of movies. In case of a tie, return the lexicographically smaller user name.
Find the movie name with the highest average rating in February 2020. In case of a tie, return the lexicographically smaller movie name.
The result format is in the following example.

 

Example 1:

Input: 
Movies table:
+-------------+--------------+
| movie_id    |  title       |
+-------------+--------------+
| 1           | Avengers     |
| 2           | Frozen 2     |
| 3           | Joker        |
+-------------+--------------+
Users table:
+-------------+--------------+
| user_id     |  name        |
+-------------+--------------+
| 1           | Daniel       |
| 2           | Monica       |
| 3           | Maria        |
| 4           | James        |
+-------------+--------------+
MovieRating table:
+-------------+--------------+--------------+-------------+
| movie_id    | user_id      | rating       | created_at  |
+-------------+--------------+--------------+-------------+
| 1           | 1            | 3            | 2020-01-12  |
| 1           | 2            | 4            | 2020-02-11  |
| 1           | 3            | 2            | 2020-02-12  |
| 1           | 4            | 1            | 2020-01-01  |
| 2           | 1            | 5            | 2020-02-17  | 
| 2           | 2            | 2            | 2020-02-01  | 
| 2           | 3            | 2            | 2020-03-01  |
| 3           | 1            | 3            | 2020-02-22  | 
| 3           | 2            | 4            | 2020-02-25  | 
+-------------+--------------+--------------+-------------+
Output: 
+--------------+
| results      |
+--------------+
| Daniel       |
| Frozen 2     |
+--------------+
Explanation: 
Daniel and Monica have rated 3 movies ("Avengers", "Frozen 2" and "Joker") but Daniel is smaller lexicographically.
Frozen 2 and Joker have a rating average of 3.5 in February but Frozen 2 is smaller lexicographically.

In [25]:
movie_rating_schema = delivery_schema = StructType([
                StructField('movie_id',IntegerType()),
                StructField('user_id',IntegerType()),
                StructField('rating',IntegerType()),
                StructField('created_at',DateType()),
                ])

In [26]:
movie_df = price_df = spark.read.option('header',True).option('inferSchema',True).format('csv')\
        .load('../data/easymedium/043_MovieRating.csv')

In [117]:
movie_df.printSchema()

root
 |-- movie_id: integer (nullable = true)
 |-- title: string (nullable = true)



In [27]:
movie_user_df = price_df = spark.read.option('header',True).option('inferSchema',True).format('csv')\
        .load('../data/easymedium/043a_MovieRating.csv')

In [82]:
movie_user_df.printSchema()

root
 |-- user_id: integer (nullable = true)
 |-- name: string (nullable = true)



In [28]:
movie_rating_df = price_df = spark.read.option('header',True).schema(movie_rating_schema).format('csv')\
        .load('../data/easymedium/043b_MovieRating.csv')

In [10]:
movie_rating_df.printSchema()

root
 |-- movie_id: integer (nullable = true)
 |-- user_id: integer (nullable = true)
 |-- rating: integer (nullable = true)
 |-- created_at: date (nullable = true)



In [80]:
movie_rating_df.createOrReplaceTempView('movierating')
movie_user_df.createOrReplaceTempView('movieuserdf')
movie_df.createOrReplaceTempView('movie')


In [138]:
spark.sql('with usr_cnt as \
                  (select user_id,cnt from \
                  (select user_id, count(*) as cnt from movierating group by user_id) x \
                  where cnt = (select max(cnt) as mcnt from (select count(*) as cnt \
                  from movierating group by user_id) as y)),\
    usr_result as (select min(name) as result from movieuserdf join usr_cnt on movieuserdf.user_id = usr_cnt.user_id), \
      movie_id as ( \
                  select movie_id,sum(rating)/count(movie_id)  as rating from \
                  (select movie_id, rating, date_format(created_at,"MM-yyyy") mmyyyy from movierating) x \
                  where mmyyyy = "02-2020" group by  movie_id), \
  movie_result as (select min(title) as result from movie join \
                  (select movie_id as ymovie_id,rating from movie_id where rating = (select max(rating) from movie_id)) y \
                  on movie_id = ymovie_id) \
                  select * from usr_result union select * from movie_result \
          ').show()


+--------+
|  result|
+--------+
|Frozen 2|
|  Daniel|
+--------+



#### 044 Restaurant Growth

Table: Customer

+---------------+---------+
| Column Name   | Type    |
+---------------+---------+
| customer_id   | int     |
| name          | varchar |
| visited_on    | date    |
| amount        | int     |
+---------------+---------+
In SQL,(customer_id, visited_on) is the primary key for this table.
This table contains data about customer transactions in a restaurant.
visited_on is the date on which the customer with ID (customer_id) has visited the restaurant.
amount is the total paid by a customer.
 

You are the restaurant owner and you want to analyze a possible expansion (there will be at least one customer every day).

Compute the moving average of how much the customer paid in a seven days window (i.e., current day + 6 days before). average_amount should be rounded to two decimal places.

Return the result table ordered by visited_on in ascending order.

The result format is in the following example.

 

Example 1:

Input: 
Customer table:
+-------------+--------------+--------------+-------------+
| customer_id | name         | visited_on   | amount      |
+-------------+--------------+--------------+-------------+
| 1           | Jhon         | 2019-01-01   | 100         |
| 2           | Daniel       | 2019-01-02   | 110         |
| 3           | Jade         | 2019-01-03   | 120         |
| 4           | Khaled       | 2019-01-04   | 130         |
| 5           | Winston      | 2019-01-05   | 110         | 
| 6           | Elvis        | 2019-01-06   | 140         | 
| 7           | Anna         | 2019-01-07   | 150         |
| 8           | Maria        | 2019-01-08   | 80          |
| 9           | Jaze         | 2019-01-09   | 110         | 
| 1           | Jhon         | 2019-01-10   | 130         | 
| 3           | Jade         | 2019-01-10   | 150         | 
+-------------+--------------+--------------+-------------+
Output: 
+--------------+--------------+----------------+
| visited_on   | amount       | average_amount |
+--------------+--------------+----------------+
| 2019-01-07   | 860          | 122.86         |
| 2019-01-08   | 840          | 120            |
| 2019-01-09   | 840          | 120            |
| 2019-01-10   | 1000         | 142.86         |
+--------------+--------------+----------------+
Explanation: 
1st moving average from 2019-01-01 to 2019-01-07 has an average_amount of (100 + 110 + 120 + 130 + 110 + 140 + 150)/7 = 122.86
2nd moving average from 2019-01-02 to 2019-01-08 has an average_amount of (110 + 120 + 130 + 110 + 140 + 150 + 80)/7 = 120
3rd moving average from 2019-01-03 to 2019-01-09 has an average_amount of (120 + 130 + 110 + 140 + 150 + 80 + 110)/7 = 120
4th moving average from 2019-01-04 to 2019-01-10 has an average_amount of (130 + 110 + 140 + 150 + 80 + 110 + 130 + 150)/7 = 142.86

In [29]:
resgrowth_schema = StructType([
                StructField('customer_id',IntegerType()),
                StructField('name',StringType()),
                StructField('visited_on',DateType()),
                StructField('amount',IntegerType())
                ])

In [30]:
resgrowth_df = price_df = spark.read.option('header',True).schema(resgrowth_schema).format('csv')\
        .load('../data/easymedium/044_RestaurantGrowth.csv')

In [226]:
resgrowth_df.printSchema()

root
 |-- customer_id: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- visited_on: date (nullable = true)
 |-- amount: integer (nullable = true)



In [153]:
resgrowth_df.show()

+-----------+-------+----------+------+
|customer_id|   name|visited_on|amount|
+-----------+-------+----------+------+
|          1|   Jhon|2019-01-01|   100|
|          2| Daniel|2019-01-02|   110|
|          3|   Jade|2019-01-03|   120|
|          4| Khaled|2019-01-04|   130|
|          5|Winston|2019-01-05|   110|
|          6|  Elvis|2019-01-06|   140|
|          7|   Anna|2019-01-07|   150|
|          8|  Maria|2019-01-08|    80|
|          9|   Jaze|2019-01-09|   110|
|          1|   Jhon|2019-01-10|   130|
|          3|   Jade|2019-01-10|   150|
+-----------+-------+----------+------+



In [185]:
window_spec = Window.orderBy('visited_on').rowsBetween(-6,0)

In [221]:
resgrowth_df.groupby('visited_on').agg(sum('amount').alias('amount'),countDistinct('visited_on').alias('rows')).orderBy('visited_on') \
    .select('visited_on','rows',\
    sum('amount').over(window_spec).alias('moving_sum'),\
    round(avg('amount').over(window_spec),2).alias('moving_avg'),
    sum('rows').over(window_spec).alias('moving_rowcnt') ).filter(col('moving_rowcnt') > 6) \
    .select('visited_on','moving_sub','moving_avg').show()

24/07/24 18:30:52 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


+----------+----------+----------+
|visited_on|moving_sub|moving_avg|
+----------+----------+----------+
|2019-01-07|       860|    122.86|
|2019-01-08|       840|     120.0|
|2019-01-09|       840|     120.0|
|2019-01-10|      1000|    142.86|
+----------+----------+----------+



In [258]:
resgrowth_df.createOrReplaceTempView('resgrowthdf')

In [279]:
spark.sql("with main as (select visited_on,sum(amount) as amount,count(distinct visited_on) as dis_rows \
                        from resgrowthdf group by visited_on), \
              second as (select visited_on, \
                        sum(amount) over(order by visited_on rows between 6 PRECEDING AND CURRENT ROW) as moving_sum, \
                        round(avg(amount) over(order by visited_on rows between 6 PRECEDING AND CURRENT ROW),2) as moving_avg, \
                        sum(dis_rows) over(order by visited_on rows between 6 PRECEDING AND CURRENT ROW) as moving_row_cnt from main) \
                        select visited_on,moving_sum,moving_avg from second where moving_row_cnt > 6 \
                        ").show()



24/07/24 19:23:03 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


+----------+----------+----------+
|visited_on|moving_sum|moving_avg|
+----------+----------+----------+
|2019-01-07|       860|    122.86|
|2019-01-08|       840|     120.0|
|2019-01-09|       840|     120.0|
|2019-01-10|      1000|    142.86|
+----------+----------+----------+



#### 045 Friend Requests Who Has the Most Friends

Table: RequestAccepted

+----------------+---------+
| Column Name    | Type    |
+----------------+---------+
| requester_id   | int     |
| accepter_id    | int     |
| accept_date    | date    |
+----------------+---------+
(requester_id, accepter_id) is the primary key (combination of columns with unique values) for this table.
This table contains the ID of the user who sent the request, the ID of the user who received the request, and the date when the request was accepted.
 

Write a solution to find the people who have the most friends and the most friends number.

The test cases are generated so that only one person has the most friends.

The result format is in the following example.

 

Example 1:

Input: 
RequestAccepted table:
+--------------+-------------+-------------+
| requester_id | accepter_id | accept_date |
+--------------+-------------+-------------+
| 1            | 2           | 2016/06/03  |
| 1            | 3           | 2016/06/08  |
| 2            | 3           | 2016/06/08  |
| 3            | 4           | 2016/06/09  |
+--------------+-------------+-------------+
Output: 
+----+-----+
| id | num |
+----+-----+
| 3  | 3   |
+----+-----+
Explanation: 
The person with id 3 is a friend of people 1, 2, and 4, so he has three friends in total, which is the most number than any others.
 

In [32]:
friends_schema = StructType([
                StructField('requester_id',IntegerType()),
                StructField('accepter_id',IntegerType()),
                StructField('accept_date',DateType())
                ])

In [33]:
friends_df = price_df = spark.read.option('header',True).schema(friends_schema).format('csv')\
        .load('../data/easymedium/045_FriendRequestsWhoHastheMostFriends.csv')

In [294]:
friends_df.printSchema()

root
 |-- requester_id: integer (nullable = true)
 |-- accepter_id: integer (nullable = true)
 |-- accept_date: date (nullable = true)



In [295]:
friends_df.show()

+------------+-----------+-----------+
|requester_id|accepter_id|accept_date|
+------------+-----------+-----------+
|           1|          2| 2016-06-03|
|           1|          3| 2016-06-08|
|           2|          3| 2016-06-08|
|           3|          4| 2016-06-09|
+------------+-----------+-----------+



In [324]:
main_df = friends_df.select(col('requester_id').alias('id') ).union(friends_df.select(col('accepter_id').alias('id') ))\
    .groupby(col('id')).agg(count('*').alias('cnt')).select(max(col('cnt')).alias('mcnt')).collect()[0][0]

In [327]:
friends_df.select(col('requester_id').alias('id') ).union(friends_df.select(col('accepter_id').alias('id') ))\
    .groupby(col('id')).agg(count('*').alias('cnt')).filter(col('cnt') == main_df).show()

+---+---+
| id|cnt|
+---+---+
|  3|  3|
+---+---+



In [328]:
friends_df.createOrReplaceTempView('friendsdf')

In [339]:
spark.sql(" with main as (select id, count(id) as cnt from  \
                         (select requester_id  as id from friendsdf union all select accepter_id as id from friendsdf) \
                         group by id ) \
                         select id, cnt from main where cnt = (select max(cnt) from main) \
            ").show()

+---+---+
| id|cnt|
+---+---+
|  3|  3|
+---+---+



 #### 046 Investments in 2016
 
 Table: Insurance

+-------------+-------+
| Column Name | Type  |
+-------------+-------+
| pid         | int   |
| tiv_2015    | float |
| tiv_2016    | float |
| lat         | float |
| lon         | float |
+-------------+-------+
pid is the primary key (column with unique values) for this table.
Each row of this table contains information about one policy where:
pid is the policyholder's policy ID.
tiv_2015 is the total investment value in 2015 and tiv_2016 is the total investment value in 2016.
lat is the latitude of the policy holder's city. It's guaranteed that lat is not NULL.
lon is the longitude of the policy holder's city. It's guaranteed that lon is not NULL.
 

Write a solution to report the sum of all total investment values in 2016 tiv_2016, for all policyholders who:

have the same tiv_2015 value as one or more other policyholders, and
are not located in the same city as any other policyholder (i.e., the (lat, lon) attribute pairs must be unique).
Round tiv_2016 to two decimal places.

The result format is in the following example.

 

Example 1:

Input: 
Insurance table:
+-----+----------+----------+-----+-----+
| pid | tiv_2015 | tiv_2016 | lat | lon |
+-----+----------+----------+-----+-----+
| 1   | 10       | 5        | 10  | 10  |
| 2   | 20       | 20       | 20  | 20  |
| 3   | 10       | 30       | 20  | 20  |
| 4   | 10       | 40       | 40  | 40  |
+-----+----------+----------+-----+-----+
Output: 
+----------+
| tiv_2016 |
+----------+
| 45.00    |
+----------+
Explanation: 
The first record in the table, like the last record, meets both of the two criteria.
The tiv_2015 value 10 is the same as the third and fourth records, and its location is unique.

The second record does not meet any of the two criteria. Its tiv_2015 is not like any other policyholders and its location is the same as the third record, which makes the third record fail, too.
So, the result is the sum of tiv_2016 of the first and last record, which is 45.

In [34]:
investment_df = price_df = spark.read.option('header',True).option('inferSchema',True).format('csv')\
        .load('../data/easymedium/046_Investmentsin2016.csv')

In [4]:
investment_df.show()

+---+--------+--------+---+---+
|pid|tiv_2015|tiv_2016|lat|lon|
+---+--------+--------+---+---+
|  1|      10|       5| 10| 10|
|  2|      20|      20| 20| 20|
|  3|      10|      30| 20| 20|
|  4|      10|      40| 40| 40|
+---+--------+--------+---+---+



In [27]:
investment_df.select('*',\
                count('*').over(Window.partitionBy('tiv_2015')).alias('tiv_2015_cnt'),\
                count('*').over(Window.partitionBy('lat','lon')).alias('loc_cnt'))\
                .filter( (col('tiv_2015_cnt') > 1) & (col('loc_cnt') ==1) ) \
                .select(round(sum(col('tiv_2016')),2).alias('tiv_2016')).show()

+--------+
|tiv_2016|
+--------+
|      45|
+--------+



In [28]:
investment_df.createOrReplaceTempView('investment')

In [31]:
query  = ''' SELECT ROUND(SUM(tiv_2016), 2) AS tiv_2016
                FROM (
                   SELECT *,
                       COUNT(*)OVER(PARTITION BY tiv_2015) AS tiv_2015_cnt,
                       COUNT(*)OVER(PARTITION BY lat, lon) AS loc_cnt
                   FROM investment
                   )t0
                WHERE tiv_2015_cnt > 1
                AND loc_cnt = 1
      '''

In [33]:
spark.sql(query).show()

+--------+
|tiv_2016|
+--------+
|      45|
+--------+



#### 047 Second Highest Salary

Table: Employee

+-------------+------+
| Column Name | Type |
+-------------+------+
| id          | int  |
| salary      | int  |
+-------------+------+
id is the primary key (column with unique values) for this table.
Each row of this table contains information about the salary of an employee.
 

Write a solution to find the second highest salary from the Employee table. If there is no second highest salary, return null (return None in Pandas).

The result format is in the following example.

 

Example 1:

Input: 
Employee table:
+----+--------+
| id | salary |
+----+--------+
| 1  | 100    |
| 2  | 200    |
| 3  | 300    |
+----+--------+
Output: 
+---------------------+
| SecondHighestSalary |
+---------------------+
| 200                 |
+---------------------+
Example 2:

Input: 
Employee table:
+----+--------+
| id | salary |
+----+--------+
| 1  | 100    |
+----+--------+
Output: 
+---------------------+
| SecondHighestSalary |
+---------------------+
| null                |
+---------------------+

In [35]:
salary_df = price_df = spark.read.option('header',True).option('inferSchema',True).format('csv')\
        .load('../data/easymedium/047_SecondHighestSalary.csv')


In [173]:
salary_df.show()

+---+------+
| id|salary|
+---+------+
|  1|   100|
|  2|   200|
|  3|   300|
+---+------+



In [174]:
window_spec = Window.orderBy("salary")

In [176]:
x.select( when( salary_df.select(count('*').alias('cnt')).first()[0] >= lit(2),salary_df.select(lead('salary').over(window_spec))\
              .first()[0]).otherwise(lit('null')).alias('second_value')).show()



+------------+
|second_value|
+------------+
|         200|
+------------+



24/07/25 20:01:39 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


In [186]:
salary_df.createOrReplaceTempView('salarydf')

In [187]:
spark.sql('with main as (select lead(salary) over(order by (salary)) as second_value from salarydf limit 1 ), \
                 cnt as (select count(*) as cnt from salarydf ) \
                        select case when cnt >= 2 then (select second_value from main) \
                        else null end as second_value from cnt \
                 ' ).show()

+------------+
|second_value|
+------------+
|        null|
+------------+



24/07/25 20:24:29 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


In [ ]:
 if(select count(*) from salarydf >=2,select * from main,"null" '